In [1]:
import pandas as pd

In [2]:
def calculate_ema(data, n): # EMA
    return data.ewm(span=n, min_periods=n).mean()

def calculate_macd(data): # MACD
    ema12 = calculate_ema(data, 12)
    ema26 = calculate_ema(data, 26)
    return ema12 - ema26

def calculate_signal(macd): # Signal
    return calculate_ema(macd, 9)

def calculate_rsi(data, period=14): # RSI
    delta = data.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

1.1. 製作個股DataFrame

In [3]:
def make_ticker_data(ticker, start_date, end_date):

    ### 取得資料 ###
    ticker_df = pd.read_excel(f'/Users/yitsung/git/MastersThesis/data/TaiwanStockData_perTicker/data/{ticker}.xlsx')
    ticker_df = ticker_df[['交易日期', '開盤', '最高', '最低', '收盤', '成交量', 
                        '融資', '外資餘額', '投信餘額', '自營商買賣超', '主力買賣超', '資金流向']]
    ticker_df = ticker_df.rename(columns={'交易日期': 'Date', '開盤': 'open', '最高': 'high', '最低': 'low', '收盤': 'close', '成交量': 'volume',
                            '融資': 'financing', '外資餘額': 'fi', '投信餘額': 'ii', '自營商買賣超': 'di', '主力買賣超': 'rp',
                            '資金流向': 'capital'})
    ### 選定日期, 特徵 ###
    ticker_df['ticker'] = ticker
    ticker_df['Date'] = pd.to_datetime(ticker_df['Date'])
    ticker_df = ticker_df[(ticker_df['Date'] >= start_date) & (ticker_df['Date'] <= end_date)].reset_index(drop=True)
    ticker_df = ticker_df[['ticker', 'Date', 'open', 'high', 'low', 'close', 'volume', 'financing', 'fi', 'ii', 'di', 'rp', 'capital']]
     
    ### 計算技術指標 ###
    ticker_df['EMA9'] = calculate_ema(ticker_df['close'], 9)
    ticker_df['EMA12'] = calculate_ema(ticker_df['close'], 12)
    ticker_df['EMA26'] = calculate_ema(ticker_df['close'], 26)
    ticker_df['MACD'] = calculate_macd(ticker_df['close'])
    ticker_df['Signal'] = calculate_signal(ticker_df['MACD'])
    ticker_df['RSI14'] = calculate_rsi(ticker_df['MACD'])

    ### 其他特徵處理 ###
    feature_to_differ = ['financing', 'fi', 'ii'] # 取differ
    ticker_df[feature_to_differ] = ticker_df[feature_to_differ].diff()
    ticker_df = ticker_df.dropna() # 這邊有dropna要注意index
    ticker_df = ticker_df.reset_index(drop=True)

    return ticker_df

In [4]:
ticker_df = make_ticker_data(ticker=2330, start_date='2020-11-17', end_date='2023-12-01')
ticker_df

,ticker,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14
0,2330,2021-01-04,530.0,540.0,528.0,536.0,39490,454.0,12463.0,-33.0,865,2342,6.0443,521.295251,518.980386,513.251221,5.729165,3.933239,84.477581
1,2330,2021-01-05,536.0,542.0,535.0,542.0,34839,-355.0,2884.0,179.0,-451,-1374,5.3592,525.437881,522.532126,515.535238,6.996887,4.619674,88.417310
2,2330,2021-01-06,555.0,555.0,541.0,549.0,55614,-256.0,5355.0,105.0,-4163,1,6.9696,530.151835,526.614084,518.179719,8.434365,5.454306,91.005801
3,2330,2021-01-07,554.0,570.0,553.0,565.0,53393,2200.0,1671.0,-75.0,2060,-402,8.7664,537.123278,532.531850,521.861371,10.670478,6.574521,93.325963
4,2330,2021-01-08,580.0,580.0,571.0,580.0,62957,-502.0,3278.0,187.0,1176,-5041,9.0658,545.700404,539.847445,526.412277,13.435169,8.026473,94.939847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,2330,2023-11-27,573.0,577.0,568.0,568.0,20322,-112.0,-2153.0,59.0,-56,-3554,4.1507,573.659169,571.840469,562.945683,8.894786,9.086560,81.069290
705,2330,2023-11-28,565.0,576.0,565.0,575.0,26932,478.0,3323.0,-98.0,687,-416,5.1624,573.927335,572.326550,563.838595,8.487955,8.966839,76.500832
706,2330,2023-11-29,578.0,579.0,570.0,574.0,27787,357.0,-180.0,55.0,-553,-2383,4.8624,573.941868,572.584004,564.591292,7.992712,8.772014,71.301362
707,2330,2023-11-30,576.0,577.0,570.0,577.0,54365,-32.0,4730.0,-68.0,-770,-155,7.5527,574.553494,573.263388,565.510455,7.752933,8.568197,68.146342


1.2. 將所有股票的Data垂直整併在一起, 並輸出CSV

In [5]:
### Top100 ###
ticker_ls = pd.read_csv('/Users/yitsung/git/MastersThesis/data/BusinessVolume_Top100_perDay.csv') 
unique_ticker = ticker_ls['Ticker'].unique() # array

TaiwanStockData = pd.DataFrame()
error_ls = []
for ticker in unique_ticker: # unique_ticker

    try: # 因為有些data會有遺失
        ticker_df = make_ticker_data(ticker=ticker, start_date='2020-11-17', end_date='2023-12-01')
        TaiwanStockData = pd.concat([TaiwanStockData, ticker_df], axis=0)
        TaiwanStockData = TaiwanStockData.reset_index(drop=True)
    
    except:
        error_ls.append(ticker)
        print(f'{ticker} failed')
        continue

TaiwanStockData.to_csv('TaiwanStockData_Top100_EMA', index=False) # error: 3025

3025 failed


In [6]:
### TopAll ###
ticker_ls = pd.read_csv('/Users/yitsung/git/MastersThesis/data/BusinessVolume_TopAll_perDay.csv') 
unique_ticker = ticker_ls['Ticker'].unique() # array

TaiwanStockData = pd.DataFrame()
error_ls = []
for ticker in unique_ticker: # unique_ticker

    try: # 因為有些data會有遺失
        ticker_df = make_ticker_data(ticker=ticker, start_date='2020-11-17', end_date='2023-12-01')
        TaiwanStockData = pd.concat([TaiwanStockData, ticker_df], axis=0)
        TaiwanStockData = TaiwanStockData.reset_index(drop=True)
    
    except:
        error_ls.append(ticker)
        print(f'{ticker} failed')
        continue

TaiwanStockData.to_csv('TaiwanStockData_TopAll_EMA', index=False) # error: 3025, 1538, 3308, 2948, 2949, 6937 

3025 failed
1538 failed
3308 failed
2948 failed
2949 failed
6937 failed
